In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import ollama
from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

In [2]:
from uuid import uuid4

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""

def reduce_messages(left: list[AnyMessage], right:list[AnyMessage])->list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [3]:
tool = TavilySearchResults(max_results=1)

In [4]:
class Agent:

    def __init__(self, model, tools, system="", chechpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_ollama)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=chechpointer,
            interrupt_before=['action']
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_ollama(self, state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)]+messages
        message = self.model.invoke(messages)
        return {"messages": [message]}
    
    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {"messages": results}

In [5]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    api_key="ollama",
    model="llama3.1",
    base_url="http://localhost:11434/v1",
)

In [6]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j4t7i5wc', 'function': {'arguments': '{"query":"SF weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 259, 'total_tokens': 282, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9ceb653f-5756-4824-adfc-67f541c1fa57-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather today'}, 'id': 'call_j4t7i5wc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282, 'input_token_details': {}, 'output_token_details': {}})]}


In [7]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    print("\n")
    print(abot.graph.get_state(thread))

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f3yxegum', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 259, 'total_tokens': 283, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-95cf596a-a903-4890-a43c-5ab33e4bc274-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_f3yxegum', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 24, 'total_tokens': 283, 'input_token_details': {}, 'output_token_details': {}})]}


StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='f2233115-f61f-41

In [8]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    print("\n")
    print(abot.graph.get_state(thread).next)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2the3vj6', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 259, 'total_tokens': 284, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4cefe72f-3ac4-44d5-b7ae-535ab97bc0e6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_2the3vj6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 25, 'total_tokens': 284, 'input_token_details': {}, 'output_token_details': {}})]}


('action',)


In [9]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    print("\n")
    print(abot.graph.get_state(thread).next)
    print("\n")
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ildh04mg', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 259, 'total_tokens': 281, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f276dbfb-67ee-4367-976e-27206cef700e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_ildh04mg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281, 'input_token_details': {}, 'output_token_details': {}})]}


('action',)


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_ildh04mg', 'type': 'tool_call'}
Back to the model!
{'messages': [Tool

In [10]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    print("\n")
    print(abot.graph.get_state(thread).next)
    print("\n")
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)
    print("\n")
    print(abot.graph.get_state(thread))

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_u8h4f1u1', 'function': {'arguments': '{"query":"SF weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 259, 'total_tokens': 281, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e4ec87d0-1e9b-4fba-a95f-b3ddc40ea7fd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_u8h4f1u1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281, 'input_token_details': {}, 'output_token_details': {}})]}


('action',)


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'call_u8h4f1u1', 'type': 'tool_call'}
Back to the model!
{'messages': [Tool

In [11]:
messages = [HumanMessage("What's the weathre in LA?")]
thread = {"configurable": {"thread_id": "2"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, chechpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    while abot.graph.get_state(thread).next:
        print("\n", abot.graph.get_state(thread), "\n")
        cmd = input("Proceed?")
        if cmd != "y":
            print("aborting")
            break
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_afswuhr8', 'function': {'arguments': '{"query":"Los Angeles weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 262, 'total_tokens': 285, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3.1', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af660794-7c0d-4253-8b84-5df8f2790ff8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather'}, 'id': 'call_afswuhr8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 262, 'output_tokens': 23, 'total_tokens': 285, 'input_token_details': {}, 'output_token_details': {}})]}

 StateSnapshot(values={'messages': [HumanMessage(content="What's the weathre in LA?", additional_kwargs={}, response_metadata={}, id='8dd243d2-7cba-4476-84e7-a